[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sapiosciences/sapio-py-tutorials/blob/master/3_custom_report.ipynb)

In [ ]:
from sapiopylib.rest.User import SapioUser
from sapiopylib.rest.DataMgmtService import DataMgmtServer
from sapiopylib.rest.pojo.CustomReport import *
from IPython.display import display, HTML
import pandas as pd
pd.set_option('display.max_columns', 100)

user = SapioUser(url="https://linux-vm:8443/webservice/api",
                 guid="3c232543-f407-4828-aae5-b33d4cd31fa7", account_name="sapio",
                 username="pyRestTest", password="Password1!", verify_ssl_cert=False)
customReportManager = DataMgmtServer.get_custom_report_manager(user)

# Custom Report (Advanced Search) API
Custom Report API offers a way for users to send complex queries about data records across a hierarchy.

"Custom Report" is the API term we use to describe the "Advanced Search" feature we present to user through the UI.

# Quick Report (Single Term)

In the first example, we will run a quick report, which is consisted with only a single term about a data record.
We will find all samples with sample type 'Blood' in the system, but only for the first 100 samples in the system.

We can obtain the data frame from report result, so it can be organized as pandas data frame and display it.

*In order for paging to actually work, you need to specify both the page size and the page number. Omitting page number causes report to return unpaged.*

In [ ]:
root_term = RawReportTerm('Sample', 'ExemplarSampleType', RawTermOperation.EQUAL_TO_OPERATOR, 'Blood', is_negated=True)
report = customReportManager.run_quick_report(root_term, page_size=100, page_number=0)
data_frame = report.get_data_frame()
display(data_frame)

# Full Custom Report (Multiple Terms)
In a more advanced report, you may want to enter multiple conditions, across a hierarchy of records.

Here's a way to get all attachments under a location '205 N George St'

Note: you need to feed in a root term to the report term parameter. The terms under the root must form a binary tree. Any terms not connected with the root will not participate in the search.

Furthermore, in order for a data type to be part of a search, you must specify at least a term, even if 

In [ ]:
term1 = RawReportTerm('VeloxLocation', 'LocationName', RawTermOperation.EQUAL_TO_OPERATOR, '205 N George St')
term2 = RawReportTerm('Attachment', 'RecordId', RawTermOperation.GREATER_THAN_OR_EQUAL_OPERATOR, '0')
root_term = CompositeReportTerm(term1, CompositeTermOperation.AND_OPERATOR, term2)
column_list = [ReportColumn('Attachment', 'FilePath', FieldType.STRING), ReportColumn('Attachment', 'RecordId', FieldType.LONG)]
request = CustomReportCriteria(column_list, root_term, page_size=1, page_number=0)
report = customReportManager.run_custom_report(request)
data_frame = report.get_data_frame()
display(data_frame)

# Inner Join, Hierarchally Unrelated Records
In Sapio 22.10, we can add **explicit joins** to records of types that are **not hierarchically related**.

In this case, let's find the sample's receipt status along with the occupancy info of the storage unit the sample "00001" is under.

Observe that in *ExplicitJoinDefinition*, the data type name outside of the join term must be set to the **Unrelated Data Type Name** in the report. In this case, storage units are not related to either sample or sample receipt via a hierarchy.

In [ ]:
sample_term = RawReportTerm('Sample', 'SampleId', RawTermOperation.EQUAL_TO_OPERATOR, '00001')
receipt_term = RawReportTerm('SampleReceipt', 'RecordId', RawTermOperation.GREATER_THAN_OPERATOR, '0')
root_term = CompositeReportTerm(sample_term, CompositeTermOperation.AND_OPERATOR, receipt_term)
storage_join_term = FieldCompareReportTerm('Sample', 'StorageLocationBarcode', RawTermOperation.EQUAL_TO_OPERATOR, 'StorageUnit', 'StorageUnitId')
column_list = [ReportColumn('Sample', 'SampleId', FieldType.STRING), ReportColumn('SampleReceipt', 'SampleReceivedRejected', FieldType.PICKLIST),
               ReportColumn('StorageUnit', 'OccupiedCount', FieldType.LONG), ReportColumn('StorageUnit', 'StorageUnitCapacity', FieldType.LONG)]
request = CustomReportCriteria(column_list=column_list, root_term=root_term, join_list=[ExplicitJoinDefinition('StorageUnit', storage_join_term)])
report = customReportManager.run_custom_report(request)
data_frame = report.get_data_frame()
display(data_frame)